# TFG: Network properties of Informal Support Networks in Mental Health

Oriol Martinez Perez

We will work with data set https://zenodo.org/record/7443141 that was created within the CoAct project (as described there). The data is stored in folder data/ within the same folder as this jupyter notebook. 

## 0. Notebook setup

In [44]:
import pandas as pd
import networkx as nx
import numpy as np
import random

In [2]:
TYPE_C_PATH = "./data/Answers_to_Microstories/answers_stories_C.pkl"
SOCIODEMOGRAPHIC_PATH = "./data/Sociodemographic_survey/answers_sociodemographic_survey.pkl"

## 1. Load some data

We will work with python, and more specifically with python pandas.

First, we load the answers of the participants to stories type C (See documentation on Zenodo, https://zenodo.org/record/7443141)

In [3]:
df_C = pd.read_pickle(TYPE_C_PATH)
df_C

0       capacitation_Teatre_Amigues                                          \
1                              yo_1  yo_2 yo_final otro_1 otro_2 otro_final   
f91cb3f                           B  None        B      C   None          C   
65fc038                           C  None        C      C   None          C   
5a20153                           C  None        C      C   None          C   
9f4d288                           C  None        C      C   None          C   
a1efa63                           A  None        A      B   None          B   
...                             ...   ...      ...    ...    ...        ...   
024f546                           C  None        C      C   None          C   
190aebe                        None  None     None   None   None       None   
4a92bfd                           C  None        C      C   None          C   
388a24f                           C  None        C      C   None          C   
c2d349a                        None  None     None   None   None       None   

0       Obrir_camí                        ... Incomprensió_1             \
1             yo_1  yo_2 yo_final otro_1  ...         otro_2 otro_final   
f91cb3f          A  None        A      C  ...           None       None   
65fc038       None  None     None   None  ...           None       None   
5a20153          A  None        A      A  ...           None       None   
9f4d288          A  None        A      A  ...           None       None   
a1efa63          A  None        A      A  ...           None       None   
...            ...   ...      ...    ...  ...            ...        ...   
024f546       None  None     None   None  ...           None       None   
190aebe       None  None     None   None  ...           None       None   
4a92bfd          C  None        C      C  ...           None       None   
388a24f       None  None     None   None  ...           None       None   
c2d349a       None  None     None   None  ...           None       None   

0       Por_situacions_socials                             \
1                         yo_1 yo_final otro_1 otro_final   
f91cb3f                   None     None   None       None   
65fc038                   None     None   None       None   
5a20153                   None     None   None       None   
9f4d288                   None     None   None       None   
a1efa63                   None     None   None       None   
...                        ...      ...    ...        ...   
024f546                   None     None   None       None   
190aebe                   None     None   None       None   
4a92bfd                   None     None   None       None   
388a24f                   None     None   None       None   
c2d349a                   None     None   None       None   

0       Reconeixement_comprensio                             
1                           yo_1 yo_final otro_1 otro_final  
f91cb3f                     None     None   None       None  
65fc038                     None     None   None       None  
5a20153                     None     None   None       None  
9f4d288                     None     None   None       None  
a1efa63                     None     None   None       None  
...                          ...      ...    ...        ...  
024f546                     None     None   None       None  
190aebe                     None     None   None       None  
4a92bfd                     None     None   None       None  
388a24f                     None     None   None       None  
c2d349a                     None     None   None       None  

[748 rows x 706 columns]

The DataFrame we loaded is a pandas MultiIndex DataFrame. We are not interested in there intermediate decisions, but only in the FINAL decisions of the participants, so we reduce the DataFrame to columns yo_final and otro_final. Particularly, we create two DataFrames with only these columns.

In [4]:
df_C.columns= df_C.columns.swaplevel(0,1)
df_C_yo   = df_C.yo_final
df_C_otro = df_C.otro_final
df_C.columns= df_C.columns.swaplevel(0,1)

df_C_yo.head(5)

,capacitation_Teatre_Amigues,Obrir_camí,Experiencia_aprenentatge,Gossos,Sanglotant,Empatia_comprensio,Encuentrobis,Ataque_ansiedad_b,Temps_aprendre,Bipolar,...,Joys_motherhood_2,Esperança,Al·lucinacions,Escoltar_temps,Germans,Suport,Realitat_ficció,Incomprensió_1,Por_situacions_socials,Reconeixement_comprensio
f91cb3f,B,A,A,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
65fc038,C,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5a20153,C,A,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9f4d288,C,A,B,C,B,A,B,C,B,None,...,None,None,None,None,None,None,None,None,None,None
a1efa63,A,A,B,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Secondly, we also load the answers of the (same) participants to the sociodemographic survey (columns documented in ./data/Sociodemographic_survey/meta_data_answers.pdf):

In [5]:
df_sociodem = pd.read_pickle(SOCIODEMOGRAPHIC_PATH)
df_sociodem

1,genere,edat,pp,pp_v,pp_v_g,pp_bf,pp_bef,pp_bel,pp_bev,pp_bec,...,p_ac_si,p_g1,p_g2,p_ass,p_prof,p_prof_si,country_alpha2,urban_rural,is_in_Catalonia,language
f91cb3f,f,4,Sí,Sí,a,m,a,a,b,b,...,None,b,m,Sí,No,None,US,None,False,en
65fc038,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,False,ca
5a20153,f,3,No,None,None,None,None,None,None,None,...,None,a,a,Sí,No,None,ES,urban,True,ca
9f4d288,m,4,No,None,None,None,None,None,None,None,...,None,b,b,Sí,Sí,apasa,ES,urban,True,ca
a1efa63,f,2,Sí,Sí,nsdmp,b,m,nsdmp,nvr,nvr,...,nvr,a,m,No,nvr,None,ES,None,False,es
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
024f546,f,1,Sí,Sí,a,m,a,nt,nsdmp,m,...,None,b,m,Sí,Sí,psm,ES,urban,True,ca
190aebe,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,False,ca
4a92bfd,m,4,Sí,Sí,m,m,b,nsdmp,nsdmp,nsdmp,...,None,a,None,No,No,None,ES,None,False,es
388a24f,f,4,Sí,Sí,a,a,a,m,b,b,...,None,a,None,None,None,None,ES,urban,True,ca


### 1.1. Dataframe exploration

In [7]:
df_C_yo.describe()

,capacitation_Teatre_Amigues,Obrir_camí,Experiencia_aprenentatge,Gossos,Sanglotant,Empatia_comprensio,Encuentrobis,Ataque_ansiedad_b,Temps_aprendre,Bipolar,...,Joys_motherhood_2,Esperança,Al·lucinacions,Escoltar_temps,Germans,Suport,Realitat_ficció,Incomprensió_1,Por_situacions_socials,Reconeixement_comprensio
count,580,399,361,332,298,281,248,238,225,204,...,21,21,21,21,20,20,20,20,20,19
unique,3,3,3,3,3,3,3,3,3,3,...,3,3,2,3,3,3,3,3,3,3
top,C,A,A,C,C,B,B,B,A,B,...,C,C,C,C,C,C,C,C,C,C
freq,391,280,152,183,203,120,87,96,104,86,...,13,14,16,9,14,11,12,12,9,11


In [22]:
df_C_yo.isna().sum()

0
capacitation_Teatre_Amigues    168
Obrir_camí                     349
Experiencia_aprenentatge       387
Gossos                         416
Sanglotant                     450
                              ... 
Suport                         728
Realitat_ficció                728
Incomprensió_1                 728
Por_situacions_socials         728
Reconeixement_comprensio       729
Length: 130, dtype: int64

**TASK**: Find out how many women and how many man are in the sample.

In [18]:
df_sociodem['genere'].value_counts()

f      366
m      155
d       11
nvr      3
Name: genere, dtype: int64

In [29]:
df_sociodem[df_sociodem['genere'] == 'f'].shape[0]

366

In [19]:
df_sociodem['country_alpha2'].value_counts()

ES    463
US     17
DE     12
MX      3
AT      3
AR      3
FI      2
BE      2
PT      2
IL      1
CH      1
FR      1
IT      1
CA      1
Name: country_alpha2, dtype: int64

In [20]:
df_sociodem.describe()

1,genere,edat,pp,pp_v,pp_v_g,pp_bf,pp_bef,pp_bel,pp_bev,pp_bec,...,p_ac_si,p_g1,p_g2,p_ass,p_prof,p_prof_si,country_alpha2,urban_rural,is_in_Catalonia,language
count,535,532,508,385,312,382,382,381,381,380,...,162,479,446,473,472,131,512,393,748,748
unique,4,7,3,3,5,5,5,6,5,5,...,4,4,4,3,3,4,14,2,2,4
top,f,4,Sí,Sí,a,a,m,nsdmp,nsdmp,nsdmp,...,m,m,m,No,No,psm,ES,urban,True,ca
freq,366,158,386,313,176,113,134,105,252,174,...,71,188,184,242,322,42,463,321,393,493


## 2. Graph creation

Get a subset of the stories that has at least 50 participants

In [26]:
df_C_yo.shape

(748, 130)

In [31]:
df_C_yo['Obrir_camí'].value_counts().sum()

399

In [6]:
stories = []
for column in df_C_yo.columns:
    if df_C_yo[column].value_counts(dropna=True).sum() > 50:
        stories.append(column)

In [7]:
subset_df = df_C_yo[stories]
subset_df

,capacitation_Teatre_Amigues,Obrir_camí,Experiencia_aprenentatge,Gossos,Sanglotant,Empatia_comprensio,Encuentrobis,Ataque_ansiedad_b,Temps_aprendre,Bipolar,...,Inclusió,Asperger_5,Paraules_màgiques,Esquizofrenia,Angustia_atemorizada_verguenza,Esgarrifós,Ingres_involuntari_2,Incomprensió_10,Colaboración_implicaciónbis,Ocultació_estigma
f91cb3f,B,A,A,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
65fc038,C,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5a20153,C,A,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9f4d288,C,A,B,C,B,A,B,C,B,None,...,None,None,None,None,None,None,None,None,None,None
a1efa63,A,A,B,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
024f546,C,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
190aebe,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4a92bfd,C,C,A,C,C,A,C,None,None,None,...,None,None,None,None,None,None,None,None,None,None
388a24f,C,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Generate a list of 3-tuples

In [32]:
def weight(ans1, ans2):
    """
    Computes weight of the edge that connects participant i and j
    """
    weight = 0
    if ans1 == 'A':
        if ans2 == 'A':
            weight = 1
        elif ans2 == 'B':
            weight = 2/3
    elif ans2 == 'B':
        if ans1 == 'A':
            weight = 2/3
        elif ans1 == 'B':
            weight = 1/3
    
    return weight

In [38]:
ind1 = subset_df.index[0]
ind2 = subset_df.index[2]

print(f'Ind 1: {ind1}, {subset_df["Obrir_camí"][ind1]}')
print(f'Ind 2: {ind2}, {subset_df["Obrir_camí"][ind2]}')

Ind 1: f91cb3f, A
Ind 2: 5a20153, A


In [34]:
print(weight(subset_df['Obrir_camí'][ind1], subset_df['Obrir_camí'][ind2]))

1


In [41]:
# generate dict that contains as keys the story names and as values the 3-tuples
elist_yo = {}
story_count = 1
for column in subset_df.columns:
    print(f'Computing weights for story number {story_count}: {column}...')
    
    elist_yo[column] = []
    for i, id1 in enumerate(subset_df.index):
        for j, id2 in enumerate(subset_df.index):
            if j > i:
                pair = [id1, id2, weight(subset_df[column][id1], subset_df[column][id2])]
                elist_yo[column].append(pair)
    
    # limit the number of stories to analyze to fasten computation
    if story_count == 1:
        break
    story_count += 1
print('Weights computation complete!')

Computing weights for story number 1: capacitation_Teatre_Amigues...
Weights computation complete!


In [42]:
# Chech we compute the weights for all indexes
if len(elist_yo['capacitation_Teatre_Amigues']) == 279378: # 748!/(746! * 2!) is 279378
    print('Correct computation')
else: 
    print('Something went wrong')

Correct computation


In [49]:
story = 'capacitation_Teatre_Amigues'
ans_lst = elist_yo[story]
for i in range(10):
    print(f'Sample {i}:')
    sample = random.choice(ans_lst)
    print(f'\tInd1: {sample[0]}, {subset_df[story][sample[0]]}')
    print(f'\tInd2: {sample[1]}, {subset_df[story][sample[1]]}')
    print(f'\tWeight: {sample[2]}')

Sample 0:
	Ind1: c094913, B
	Ind2: 02b634c, B
	Weight: 0.3333333333333333
Sample 1:
	Ind1: ac83cfc, B
	Ind2: 159fdad, C
	Weight: 0
Sample 2:
	Ind1: 8470e07, None
	Ind2: fd01af9, C
	Weight: 0
Sample 3:
	Ind1: 679fa36, B
	Ind2: ff10184, B
	Weight: 0.3333333333333333
Sample 4:
	Ind1: ea052b5, C
	Ind2: 0ff20c2, C
	Weight: 0
Sample 5:
	Ind1: f0d3a29, B
	Ind2: db0051d, C
	Weight: 0
Sample 6:
	Ind1: a12b731, C
	Ind2: 53deb5c, B
	Weight: 0
Sample 7:
	Ind1: daa4f97, B
	Ind2: 905df17, None
	Weight: 0
Sample 8:
	Ind1: 5b603bb, C
	Ind2: 1f68249, C
	Weight: 0
Sample 9:
	Ind1: 3d3d373, None
	Ind2: 1b88f70, B
	Weight: 0
